# Getting Started - RocketPy in Colab


We start by setting up our environment. To run this notebook, we will need:

- RocketPy
- Data files (we will download RocketPy's data from github)

Therefore, let's run the following lines of code:

In [ ]:
!pip install rocketpy
!curl -o NACA0012-radians.txt https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/airfoils/NACA0012-radians.txt
!curl -o Cesaroni_M1670.eng https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/motors/cesaroni/Cesaroni_M1670.eng
!curl -o powerOffDragCurve.csv https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/rockets/calisto/powerOffDragCurve.csv
!curl -o powerOnDragCurve.csv https://raw.githubusercontent.com/RocketPy-Team/RocketPy/master/data/rockets/calisto/powerOnDragCurve.csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 453.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.5/372.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.5 MB/s eta 0:00:00
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=7b8624415fb58c12fd8f5143d8932e081f6290bf1edfd99c00a2eb60aa65c4ec
  Stored in directory: /root/.cache/pip/wheels/83/ee/f2/65cecfd948f1429ead035fd6d56bc6bd6574a636ddc4d65cbd
Successfully built simplekml
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1052  100  1052    0     0   2131      0 --:--:-- --:--:-- --:--:--  2129
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dl

Now we can start!

Here we go through a simplified rocket trajectory simulation to get you started. Let's start by importing the rocketpy module.


In [ ]:
from rocketpy import Environment, Flight, Rocket, SolidMotor

It is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.


In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

## Setting Up a Simulation


### Creating an Environment for Spaceport America



The `Environment` class is used to define the atmosphere, the winds, and the gravity models.

You can find more information about the `Environment` class in the [Environment Class Usage Docs](https://docs.rocketpy.org/en/latest/notebooks/environment/environment_class_usage.html).

In [ ]:
env = Environment(latitude=-21.890000, longitude=-49.030000, elevation=422)  #Cidade de Lacanga, São Paulo

To get weather data from the GFS forecast, available online, we run the following lines.

First, we set tomorrow's date.


In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

env.set_date(
    (tomorrow.year, tomorrow.month, tomorrow.day, 12)
)  # Hour given in UTC time

Then, we tell env to use a GFS forecast to get the atmospheric conditions for flight.

Don't mind the warning, it just means that not all variables, such as wind speed or atmospheric temperature, are available at all altitudes given by the forecast.


In [ ]:
env.set_atmospheric_model(type="Forecast", file="GFS")

We can see what the weather will look like by calling the info method!


In [ ]:
env.info()


Gravity Details

Acceleration of gravity at surface level:    9.7861 m/s²
Acceleration of gravity at  79.344 km (ASL): 9.5471 m/s²


Launch Site Details

Launch Date: 2025-08-24 12:00:00 UTC
Launch Site Latitude: -21.89000°
Launch Site Longitude: -49.03000°
Reference Datum: SIRGAS2000
Launch Site UTM coordinates: 83450.27 W    7573880.53 S
Launch Site UTM zone: 23K
Launch Site Surface Elevation: 466.8 m


Atmospheric Model Details

Atmospheric Model Type: Forecast
Forecast Maximum Height: 79.344 km
Forecast Time Period: from 2025-08-23 06:00:00 to 2025-09-08 06:00:00 utc
Forecast Hour Interval: 3 hrs
Forecast Latitude Range: From -90.0° to 90.0°
Forecast Longitude Range: From 0.0° to 359.75°

Surface Atmospheric Conditions

Surface Wind Speed: 2.90 m/s
Surface Wind Direction: 153.45°
Surface Wind Heading: 333.45°
Surface Pressure: 964.60 hPa
Surface Temperature: 295.14 K
Surface Air Density: 1.139 kg/m³
Surface Speed of Sound: 344.40 m/s


Earth Model Details

Earth Radius at Launch s

### Creating a Motor

A solid rocket motor is used in this case. To create a motor, the SolidMotor class is used and the required arguments are given.

The SolidMotor class requires the user to have a thrust curve ready. This can come either from a .eng file for a commercial motor, such as below, or a .csv file from a static test measurement.

Besides the thrust curve, other parameters such as grain properties and nozzle dimensions must also be given.

See [Solid Motor Class Usage Docs](https://docs.rocketpy.org/en/latest/user/motors/solidmotor.html) for more information.


In [ ]:
Serena = SolidMotor(
    thrust_source="Q2_04.eng",  # Caminho do arquivo .eng
    dry_mass= 0.58684,  # kg - massa do motor sem propelente
    dry_inertia= (0.00741858, 0.00741858, 0.00026227), # kg·m² - momento de inércia do motor sem propelente (eixos I, I, Z)
    nozzle_radius= 24.5 / 1000,  # m - raio da saída do bocal (x mm)
    grain_number= 4,  # número de grãos de propelente
    grain_density= 1000.748 ,  # kg/m³ - densidade do propelente
    grain_outer_radius= 30 / 1000,  # m - raio externo do grão (x mm)
    grain_initial_inner_radius= 17 / 1000,  # m - raio interno inicial (oco) do grão (x mm)
    grain_initial_height= 50 / 1000,  # m - altura de cada grão (x mm)
    grain_separation= 3.23 / 1000,  # m - espaço entre os grãos (x mm)
    grains_center_of_mass_position= 0.1045,  # m - posição do centro de massa dos grãos
    center_of_dry_mass_position= 0.175,  # m - posição do CM do motor sem propelente
    nozzle_position=0,
    burn_time = 1.23,
    throat_radius= 9.5 / 1000,  # m - raio da garganta do bocal (x mm)
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

**Pay special attention to *position* related parameters:**
More details on [Positions and Coordinate Systems](https://docs.rocketpy.org/en/latest/user/positions.html)


To see what our thrust curve looks like, along with other import properties, we invoke the info method yet again. You may try the all_info method if you want more information all at once!


In [ ]:
Serena.info()

### Creating a Rocket


A rocket is composed of several components. Namely, we must have a motor (good thing we have the Pro75M1670 ready), a couple of aerodynamic surfaces (nose cone, fins and tail) and parachutes (if we are not launching a missile).

You can find more information about the `Rocket` class in the [Rocket Class Usage Docs](https://docs.rocketpy.org/en/latest/user/rocket.html).

Let's start by initializing our rocket, named Calisto, entering inertia properties, some dimensions and drag curves.

**Pay special attention to *position* related parameters:**
More details on [Positions and Coordinate Systems](https://docs.rocketpy.org/en/latest/user/positions.html)



In [ ]:
calisto = Rocket(
    radius=0.039,
    mass=2.222,
    inertia=(0.00462543, 0.11515348, 0.11516779),
    power_off_drag="Quark2powerOffDragCurve.csv",
    power_on_drag="Quark2powerOnDragCurve.csv",
    center_of_mass_without_motor=0.42,
    coordinate_system_orientation="nose_to_tail",
)

rail_buttons = calisto.set_rail_buttons(
    upper_button_position=0.33,
    lower_button_position=0.637,
    angular_position=45,
)

To add the motor to our rocket we need only inform what motor we are adding (Pro75M1670) and inform the position, in meters, of the motor's nozzle exit area relative to the previously defined coordinate system.


In [ ]:
calisto.add_motor(Serena, position=0.896)

#### Adding Aerodynamic Surfaces


Now we define the aerodynamic surfaces. They are really straight forward with special attention needed only for the position values. Here is a quick guide:

- The positions given **must** be relative to the same coordinate system as the rockets;
- Position of the Nosecone refers to the tip of the nose;
- Position of fins refers to the point belonging to the root chord which is highest in the rocket coordinate system;
- Position of the tail the point belonging to the tail which is highest in the rocket coordinate system.

See more details in [Positions and Coordinate Systems](https://docs.rocketpy.org/en/latest/user/positions.html)

In [ ]:
nose_cone = calisto.add_nose(length=0.120, kind="elliptical", position=-0.1)

fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.110,
    tip_chord=0.080,
    span=0.120,
    position=0.728,
    cant_angle=20,
    airfoil=("NACA0012-radians.txt", "radians"),
)

tail = calisto.add_tail(
    top_radius=0.039, bottom_radius=0.0275, length=0.058, position=0.831
)

To see all information regarding the rocket we just defined we run:


In [ ]:
calisto.all_info()

#### Adding Parachutes


Finally, we have parachutes! Calisto will have two parachutes, Drogue and Main. The Drogue parachute will open at apogee while the Main parachute will open at 800m above ground level

For more details see [Adding Parachutes](https://docs.rocketpy.org/en/latest/user/rocket.html#adding-parachutes)

In [ ]:
Main = calisto.add_parachute(
    "Main",
    cd_s=2.020,
    trigger=800,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = calisto.add_parachute(
    "Drogue",
    cd_s=1.0,
    trigger="apogee",
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)



Just be careful if you run this last cell multiple times! If you do so, your rocket will end up with lots of parachutes which activate together, which may cause problems during the flight simulation. We advise you to re-run all cells which define our rocket before running this, preventing unwanted old parachutes. Alternatively, you can run the following lines to remove parachutes.

```python
Calisto.parachutes.remove(Drogue)
Calisto.parachutes.remove(Main)
```


## Simulating a Flight

Simulating a flight trajectory is as simple as initializing a Flight class object givin the rocket and environnement set up above as inputs. The launch rail inclination and heading are also given here.


In [ ]:
test_flight = Flight(
    rocket=calisto, environment=env, rail_length=5.2, inclination=85, heading=0
)

## Analyzing the Results

RocketPy gives you many plots, thats for sure! They are divided into sections to keep them organized. Alternatively, see the Flight class documentation to see how to get plots for specific variables only, instead of all of them at once.


In [ ]:
test_flight.all_info()
# Plot altitude ASL
test_flight.plot(quantities=['altitude'])

Export Flight Trajectory to a .kml file so it can be opened on Google Earth


In [ ]:
test_flight.export_kml(
    file_name="trajectory.kml",
    extrude=True,
    altitude_mode="relative_to_ground",
)

## Using Simulation for Design

Here, we go through a couple of examples which make use of RocketPy in cool ways to help us design our rocket.


### Apogee as a Function of Mass

This one is a classic one! We always need to know how much our rocket's apogee will change when our payload gets heavier.


In [ ]:
from rocketpy.utilities import apogee_by_mass

apogee_by_mass(flight=test_flight, min_mass=5, max_mass=20, points=10, plot=True)

### Out of Rail Speed as a Function of Mass

Lets make a really important plot. Out of rail speed is the speed our rocket has when it is leaving the launch rail. This is crucial to make sure it can fly safely after leaving the rail. A common rule of thumb is that our rocket's out of rail speed should be 4 times the wind speed so that it does not stall and become unstable.


In [ ]:
from rocketpy.utilities import liftoff_speed_by_mass

liftoff_speed_by_mass(flight=test_flight, min_mass=5, max_mass=20, points=10, plot=True)

### Dynamic Stability Analysis

Ever wondered how static stability translates into dynamic stability? Different static margins result in different dynamic behavior, which also depends on the rocket's rotational inertial.

Let's make use of RocketPy's helper class called Function to explore how the dynamic stability of Calisto varies if we change the fins span by a certain factor.


### Characteristic Frequency Calculation

Here we analyse the characteristic frequency of oscillation of our rocket just as it leaves the launch rail. Note that when we ran test_flight.all_info(), one of the plots already showed us the frequency spectrum of our flight. Here, however, we have more control of what we are plotting.
